# Krippendorff alpha Python Implementation

Source code:
https://github.com/grrrr/krippendorff-alpha

In [1]:
import pandas as pd
import numpy as np

def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.

Let's create a dummy set of workers' assessments following format specified in function definition.

(e.g. rows = workers ; columns = items ; values = float/integer)

In [9]:
worker1 = [1,1,1,0,0,1,1]
worker2 = [1,1,1,0,0,1,1]
worker3 = [1,1,1,0,0,1,1] #perfect
worker4 = [1,1,1,0,0,1,0] #high reliability
worker5 = [0,0,1,1,1,1,1] #low reliability

perfect = krippendorff_alpha([worker1,worker2,worker3])
high = krippendorff_alpha([worker1,worker2,worker4])
low = krippendorff_alpha([worker1,worker2,worker5])

print("perfect: {:.2f}\nhigh discrepanc: {:.2f}\nlow discrepancy: {:.2f}".format(perfect,high,low))

perfect: 1.00
high discrepanc: 0.80
low discrepancy: 0.11


If reading from csv using pandas, load data as df and then convert to list.

Data in csv file should also be formatted as rows = workers and columns = items evaluated.

In [66]:
path = 'file_path'
df = pd.read_csv(path)
d = df.values.tolist()
krippendorff_alpha(d)